<a href="https://colab.research.google.com/github/mz-zarei/Trajectory_Analysis/blob/main/INTERACTION_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>